In [ ]:
import datetime
import itertools
import numpy as np
import pandas as pd
from ruamel.yaml import YAML
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10, Category20
from bokeh.resources import INLINE
import panel as pn

pn.extension()

In [ ]:
now = datetime.datetime.now()
dt_string = now.strftime("%d-%m-%Y")

In [ ]:
header = pn.pane.Markdown(f"""
# COVID-19 -- Worldwide

**NUEVO!** -- Datos de Argentina disponibles en [epassaro.github.io/covid-19/ar](https://epassaro.github.io/covid-19/ar).

- Updates automatically at 00:00 UTC. **Last update: ** {dt_string}.
- Data downloaded from [John Hopkins University repository](https://github.com/CSSEGISandData/COVID-19).
- Code released [here](https://github.com/epassaro/covid-19) under the [GNU GPLv3 License](https://raw.githubusercontent.com/epassaro/covid-19/master/LICENSE).
""", width=800, margin=(0, 0, 10, 0)
)

In [ ]:
fname = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

In [ ]:
yaml = YAML()

with open('plots.yml') as f:
    countries = yaml.load(f)

In [ ]:
def get_top_ten(fname, how='total'):
    df = pd.read_csv(fname)
    df = df.drop(columns=['Province/State', 'Lat', 'Long'])
    df = df.groupby(by=['Country/Region']).sum()

    if how == 'daily':
        df['diff'] = df.iloc[:,-1] - df.iloc[:,-2]
        df = df.sort_values(by='diff', ascending=False)

    else:
        df = df.iloc[:,-1]
        df = df.sort_values(ascending=False)

    top_ten = df.head(10).index.tolist()

    return top_ten

In [ ]:
def create_table(fname, countries, key):

    map_names = {'US': 'United States',
                 'Korea, South': 'South Korea'}

    df = pd.read_csv(fname)
    df = df.drop(columns=['Province/State', 'Lat', 'Long'])
    df = df.groupby(by=['Country/Region']).sum()    
    df = df.transpose()

    df.columns.name = None
    df.index.name = 'date'
    df.index = pd.to_datetime(df.index)

    df_list = []
    for country in countries[key]:

        d = df[country]
        n = d.diff()

        n[n < 0] = 0 # remove negative diffs (dirty)
        n[n >= 5*n.std() ] = 0 # remove outliers (dirty)

        r = n.rolling(7).mean()
        
        n.name = f'diff{map_names.get(country, country)}'       
        r.name = f'roll{map_names.get(country, country)}'   

        df_list.append(d)
        df_list.append(n)
        df_list.append(r)


    df = pd.concat(df_list, axis=1)
    df = df.rename(columns=map_names)

    df = df.reset_index()
    df['datestr'] = df['date'].map(lambda x: x.strftime('%d-%m-%Y'))
    df = df.set_index('date')

    return df

In [ ]:
def make_plots(df, name):

    n = len(df.columns) // 3

    if n <= 10:
        palette = itertools.cycle(Category10[n])

    else:
        palette = itertools.cycle(Category20[n])

    source = ColumnDataSource(df)
    p = figure(plot_width=680, plot_height=420, name='Total Cases', x_axis_type='datetime')
    q = figure(plot_width=680, plot_height=420, name='Daily Cases', x_axis_type='datetime')

    hovers = []
    for country in df[ [c for c in df.columns if c[:4] not in ['date', 'diff', 'roll']] ]:

        color = next(palette)
        p.line(x='date', y=country, source=source,
               line_width=1.5, 
               legend_label=country, 
               color=color)

        p.circle(x='date', y=country, source=source,
                 legend_label=country,
                 color=color,
                 name=country) # Tooltip are shown only on points

        q.vbar(x='date', top=f'diff{country}', source=source,
                 legend_label=country,
                 width=datetime.timedelta(days=1),
                 line_width=0,
                 color=color,
                 alpha=0.5,
                 name=country)

        q.line(x='date', y=f'roll{country}', source=source,
                 legend_label=country,
                 line_width=1.5,
                 color=color,
                 alpha=0.75,
                 name=country)

        TOOLTIPS = [('Country', country),
                    ('Date', '@{datestr}'),
                    ('New cases', f'@{{diff{country}}}{{0,0}}'),
                    ('7-d mov. average', f'@{{roll{country}}}{{0,0.0}}'),
                    ('Total cases', f'@{{{country}}}{{0,0}}')] # Double curly braces for names containing whitespaces

        hover = HoverTool(tooltips=TOOLTIPS, names=[country])
        hovers.append(hover)

    p.add_tools(*hovers)
    p.legend.click_policy = 'hide'
    p.legend.location = 'top_left'
    p.legend.label_text_font_size = '8pt'
    p.yaxis.formatter = NumeralTickFormatter(format='0,0')
    p.xaxis.axis_label = "Date"
    p.yaxis.axis_label = "Total cases"
    p.title.text = f"Cumulative confirmed COVID-19 cases ({name})"

    q.add_tools(*hovers)
    q.legend.click_policy = 'hide'
    q.legend.location = 'top_left'
    q.legend.label_text_font_size = '8pt'
    q.yaxis.formatter = NumeralTickFormatter(format='0,0')
    q.xaxis.axis_label = "Date"
    q.yaxis.axis_label = "Daily cases"
    q.title.text = f"Daily new confirmed COVID-19 cases ({name})"

    return pn.Tabs(q, p, name=name, margin=(10, 20, 0, 20))

In [ ]:
countries['Top Daily'] = get_top_ten(fname, how='daily')
countries['Top Total'] = get_top_ten(fname)

countries.move_to_end('Top Daily', last=False)
countries.move_to_end('Top Total', last=False)

In [ ]:
_ = [ make_plots(create_table(fname, countries, k), k) for k in countries ]

In [ ]:
tabs = pn.Tabs(*_)

In [ ]:
pn.Column(header, 
          tabs, 
          margin=(20, 0, 40, 40)
         ).save('index.html', 
                title='COVID-19 - Worldwide', 
                resources=INLINE)